### Clean up first

In [1]:
import sys.process._
"rm -rf /tmp/hudi/".!!
"mkdir -p /tmp/hudi".!!
lazy val result3 = "tree -a /tmp/hudi/".!!
println(result3)


/tmp/hudi/

0 directories, 0 files



import sys.process._
res1_1: String = ""
res1_2: String = ""
result3: String = [lazy]

### Setup project

In [2]:
import $ivy.`org.apache.hudi:hudi-spark3.3-bundle_2.12:1.0.1`
// import $ivy.`org.apache.hudi:hudi-common:1.0.0`
import $ivy.`org.apache.spark:spark-sql_2.12:3.3.2`
import $ivy.`org.apache.spark:spark-hive_2.12:3.3.2`
import $cp.`hudi/CustomMergeIntoConnector.jar`

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord

val spark = SparkSession.builder()
  .appName("HudiLocalSession")
  .master("local[*]")  // Runs on local machine
  .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
  .config("spark.kryo.registrator", "org.apache.spark.HoodieKryoRegistrar")
  .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
  .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar")
  .config("spark.driver.extraJavaOptions", "-Dscala.repl.maxprintstring=0")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark with Hudi is ready!")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


25/03/03 23:14:23 WARN Utils: Your hostname, DESKTOP-M94RUSC resolves to a loopback address: 127.0.1.1; using 172.17.75.227 instead (on interface eth0)
25/03/03 23:14:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/03 23:14:24 INFO SparkContext: Running Spark version 3.3.2
25/03/03 23:14:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/03 23:14:24 INFO ResourceUtils: ==============================================================
25/03/03 23:14:24 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/03 23:14:24 INFO ResourceUtils: ==============================================================
25/03/03 23:14:24 INFO SparkContext: Submitted application: HudiLocalSession
25/03/03 23:14:24 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024

import $ivy.$
import $ivy.$
import $ivy.$
import $cp.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord
spark: SparkSession = org.apache.spark.sql.SparkSession@79026d5

### Index data

In [3]:
import spark.implicits._

val tableName = "trips_table_index"
val basePath = "file:///tmp/hudi/hudi_indexed_table"

val columns = Seq("ts","uuid","rider","driver","fare","city")
val data =
  Seq((1695159649087L,"334e26e9-8355-45cc-97c6-c31daf0df330","rider-A","driver-K",19.10,"san_francisco"),
    (1695091554788L,"e96c4396-3fad-413a-a942-4cb36106d721","rider-C","driver-M",27.70 ,"san_francisco"),
    (1695046462179L,"9909a8b1-2d15-4d3d-8ec9-efc48c536a00","rider-D","driver-L",33.90 ,"san_francisco"),
    (1695516137016L,"e3cf430c-889d-4015-bc98-59bdce1e530c","rider-F","driver-P",34.15,"sao_paulo"    ),
    (1695115999911L,"c8abbe79-8d89-47ea-b4ce-4d224bae5bfa","rider-J","driver-T",17.85,"chennai"));

var inserts = spark.createDataFrame(data).toDF(columns:_*)
inserts.write.format("hudi").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  option("hoodie.write.record.merge.mode", "COMMIT_TIME_ORDERING").
  option("hoodie.datasource.write.hive_style_partitioning", "true"). // Force hive style dir naming for partitions.
  option(RECORDKEY_FIELD_OPT_KEY, "uuid").
  mode(Overwrite).
  save(basePath)

cell3.sc:22: value RECORDKEY_FIELD_OPT_KEY in class KeyGeneratorOptions is deprecated
  option(RECORDKEY_FIELD_OPT_KEY, "uuid").
         ^


# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


import spark.implicits._
tableName: String = "trips_table_index"
basePath: String = "file:///tmp/hudi/hudi_indexed_table"
columns: Seq[String] = List("ts", "uuid", "rider", "driver", "fare", "city")
data: Seq[(Long, String, String, String, Double, String)] = List(
  (
    1695159649087L,
    "334e26e9-8355-45cc-97c6-c31daf0df330",
    "rider-A",
    "driver-K",
    19.1,
    "san_francisco"
  ),
  (
    1695091554788L,
    "e96c4396-3fad-413a-a942-4cb36106d721",
    "rider-C",
    "driver-M",
    27.7,
    "san_francisco"
  ),
  (
    1695046462179L,
    "9909a8b1-2d15-4d3d-8ec9-efc48c536a00",
    "rider-D",
    "driver-L",
    33.9,
    "san_francisco"
  ),
  (
    1695516137016L,
    "e3cf430c-889d-4015-bc98-59bdce1e530c",
    "rider-F",
    "driver-P",
    34.15,
    "sao_paulo"
  ),
  (
    1695115999911L,
    "c8abbe79-8d89-47ea-b4ce-4d224bae5bfa",
    "rider-J",
    "driver-T",
    17.85,
...
inserts: DataFrame = [ts: bigint, uuid: string ... 4 more fields]

In [4]:
lazy val result = "tree -a /tmp/hudi/hudi_indexed_table".!!
println(result)

/tmp/hudi/hudi_indexed_table
├── .hoodie
│   ├── .aux
│   │   └── .bootstrap
│   │       ├── .fileids
│   │       └── .partitions
│   ├── .hoodie.properties.crc
│   ├── .index_defs
│   │   ├── .index.json.crc
│   │   └── index.json
│   ├── .schema
│   ├── .temp
│   ├── hoodie.properties
│   ├── metadata
│   │   ├── .hoodie
│   │   │   ├── .aux
│   │   │   │   └── .bootstrap
│   │   │   │       ├── .fileids
│   │   │   │       └── .partitions
│   │   │   ├── .hoodie.properties.crc
│   │   │   ├── .schema
│   │   │   ├── .temp
│   │   │   ├── hoodie.properties
│   │   │   └── timeline
│   │   │       ├── .00000000000000000.deltacommit.inflight.crc
│   │   │       ├── .00000000000000000.deltacommit.requested.crc
│   │   │       ├── .00000000000000000_20250303231429171.deltacommit.crc
│   │   │       ├── .00000000000000001.deltacommit.inflight.crc
│   │   │       ├── .00000000000000001.deltacommit.requested.crc
│   │   │       ├── .00000000000000001_20250303231429371.deltacommit.crc
│   │ 

result: String = [lazy]

In [5]:
// Create record index and secondary index for the table
spark.sql(s"CREATE TABLE IF NOT EXISTS hudi_indexed_table USING hudi LOCATION '$basePath'")
// Create bloom filter expression index on driver column
spark.sql(s"CREATE INDEX idx_bloom_driver ON hudi_indexed_table USING bloom_filters(driver) OPTIONS(expr='identity')");
// It would show bloom filter expression index 
spark.sql(s"SHOW INDEXES FROM hudi_indexed_table").show();
// Query on driver column would prune the data using the idx_bloom_driver index
spark.sql(s"SELECT uuid, rider FROM hudi_indexed_table WHERE driver = 'driver-S'").show();

+--------------------+---------------+----------+
|          index_name|       col_name|index_type|
+--------------------+---------------+----------+
|        column_stats|   column_stats|          |
|     partition_stats|partition_stats|          |
|expr_index_idx_bl...|  bloom_filters|    driver|
+--------------------+---------------+----------+

+----+-----+
|uuid|rider|
+----+-----+
+----+-----+



res5_0: DataFrame = []
res5_1: DataFrame = []

* No driver-S, so bloom filter should filter out the query fast.

* Note that the merge mode is COMMIT_TIME_ORDERING, which means: 
    * (1) COMMIT_TIME_ORDERING: use commit time to merge records, i.e., the record from later commit overwrites the earlier record with the same key. 
    * https://hudi.apache.org/docs/record_merger/

In [6]:
lazy val result = "tree -a /tmp/hudi/hudi_indexed_table".!!
println(result)

/tmp/hudi/hudi_indexed_table
├── .hoodie
│   ├── .aux
│   │   └── .bootstrap
│   │       ├── .fileids
│   │       └── .partitions
│   ├── .hoodie.properties.crc
│   ├── .index_defs
│   │   ├── .index.json.crc
│   │   └── index.json
│   ├── .schema
│   ├── .temp
│   ├── hoodie.properties
│   ├── metadata
│   │   ├── .hoodie
│   │   │   ├── .aux
│   │   │   │   └── .bootstrap
│   │   │   │       ├── .fileids
│   │   │   │       └── .partitions
│   │   │   ├── .heartbeat
│   │   │   ├── .hoodie.properties.crc
│   │   │   ├── .schema
│   │   │   ├── .temp
│   │   │   ├── hoodie.properties
│   │   │   └── timeline
│   │   │       ├── .00000000000000000.deltacommit.inflight.crc
│   │   │       ├── .00000000000000000.deltacommit.requested.crc
│   │   │       ├── .00000000000000000_20250303231429171.deltacommit.crc
│   │   │       ├── .00000000000000001.deltacommit.inflight.crc
│   │   │       ├── .00000000000000001.deltacommit.requested.crc
│   │   │       ├── .00000000000000001_2025030323142

result: String = [lazy]

* Here we can find the filter in the metadata dir.

In [7]:
"ls -l /tmp/hudi/hudi_indexed_table/.hoodie/metadata".!!

res7: String = """total 16
drwxr-xr-x 2 ryan ryan 4096 Mar  3 23:14 column_stats
drwxr-xr-x 2 ryan ryan 4096 Mar  3 23:14 expr_index_idx_bloom_driver
drwxr-xr-x 2 ryan ryan 4096 Mar  3 23:14 files
drwxr-xr-x 2 ryan ryan 4096 Mar  3 23:14 partition_stats
"""

In [8]:
val df_1 = spark.read.format("hudi").load(basePath)
df_1.withColumn("human_readable_ts", from_unixtime(col("ts")/1000, "yyyy-MM-dd")).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+-----------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|human_readable_ts|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+-----------------+
|  20250303231427781|20250303231427781...|e96c4396-3fad-413...|    city=san_francisco|75a7500e-714d-411...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M| 27.7|san_francisco|       2023-09-19|
|  20250303231427781|20250303231427781...|9909a8b1-2d15-4d3...|    city=san_francisco|75a7500e-714d-411...|1695046462179|9909a8b1-2d15-4d3...|rider-D|driver-L| 33.9|san_francisco|       2023-09-18|
|  2025030

df_1: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [9]:
// Create column stat expression index on ts column
spark.sql(s"CREATE INDEX idx_column_ts ON hudi_indexed_table USING column_stats(ts) OPTIONS(expr='from_unixtime', format = 'yyyy-MM-dd')");
// Shows both expression indexes 
spark.sql(s"SHOW INDEXES FROM hudi_indexed_table").show();
// Query on ts column would prune the data using the idx_column_ts index
spark.sql(s"SELECT * FROM hudi_indexed_table WHERE from_unixtime(ts/1000, 'yyyy-MM-dd') = '2023-09-24'").show();

+--------------------+---------------+----------+
|          index_name|       col_name|index_type|
+--------------------+---------------+----------+
|        column_stats|   column_stats|          |
|     partition_stats|partition_stats|          |
|expr_index_idx_co...|   column_stats|        ts|
|expr_index_idx_bl...|  bloom_filters|    driver|
+--------------------+---------------+----------+

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|     city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+---------+
|  20250303231427781|20250303231427781...|e3cf430c-889d-401...|        

res9_0: DataFrame = []

In [10]:
// To create secondary index, first create the record index
spark.sql(s"SET hoodie.metadata.record.index.enable=true");
spark.sql(s"CREATE INDEX record_index ON hudi_indexed_table (uuid)");
// Create secondary index on rider column
spark.sql(s"CREATE INDEX idx_rider ON hudi_indexed_table (rider)");

// Expression index and secondary index should show up
spark.sql(s"SHOW INDEXES FROM hudi_indexed_table").show();
// Query on rider column would leverage the secondary index idx_rider
spark.sql(s"SELECT * FROM hudi_indexed_table WHERE rider = 'rider-E'").show();

+--------------------+---------------+----------+
|          index_name|       col_name|index_type|
+--------------------+---------------+----------+
|        record_index|   record_index|          |
|        column_stats|   column_stats|          |
|     partition_stats|partition_stats|          |
|secondary_index_i...|secondary_index|     rider|
|expr_index_idx_co...|   column_stats|        ts|
|expr_index_idx_bl...|  bloom_filters|    driver|
+--------------------+---------------+----------+

+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name| ts|uuid|rider|driver|fare|city|
+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
+-------------------+--------------------+------------------+----------------------+----------

res10_0: DataFrame = [key: string, value: string]
res10_1: DataFrame = []
res10_2: DataFrame = []

In [11]:
lazy val result = "tree -a /tmp/hudi/hudi_indexed_table".!!
println(result)

/tmp/hudi/hudi_indexed_table
├── .hoodie
│   ├── .aux
│   │   └── .bootstrap
│   │       ├── .fileids
│   │       └── .partitions
│   ├── .hoodie.properties.crc
│   ├── .index_defs
│   │   ├── .index.json.crc
│   │   └── index.json
│   ├── .schema
│   ├── .temp
│   ├── hoodie.properties
│   ├── metadata
│   │   ├── .hoodie
│   │   │   ├── .aux
│   │   │   │   └── .bootstrap
│   │   │   │       ├── .fileids
│   │   │   │       └── .partitions
│   │   │   ├── .heartbeat
│   │   │   ├── .hoodie.properties.crc
│   │   │   ├── .schema
│   │   │   ├── .temp
│   │   │   ├── hoodie.properties
│   │   │   └── timeline
│   │   │       ├── .00000000000000000.deltacommit.inflight.crc
│   │   │       ├── .00000000000000000.deltacommit.requested.crc
│   │   │       ├── .00000000000000000_20250303231429171.deltacommit.crc
│   │   │       ├── .00000000000000001.deltacommit.inflight.crc
│   │   │       ├── .00000000000000001.deltacommit.requested.crc
│   │   │       ├── .00000000000000001_2025030323142

result: String = [lazy]

In [12]:
// Update a record and query the table based on indexed columns
spark.sql(s"SET hoodie.datasource.write.row.writer.enable=false");
spark.sql(s"UPDATE hudi_indexed_table SET rider = 'rider-B', driver = 'driver-N', ts = '1697516137' WHERE rider = 'rider-A'");
// Sorry, but I am not sure whats the deal with org.apache.hudi.exception.HoodieMetadataException: 
// Failed to get expression index updates for partition expr_index_idx_column_ts.

25/03/03 23:14:41 ERROR Executor: Exception in task 0.0 in stage 107.0 (TID 264)
org.apache.hudi.exception.HoodieException: unable to read next record from parquet file 
	at org.apache.hudi.common.util.ParquetReaderIterator.hasNext(ParquetReaderIterator.java:54)
	at org.apache.hudi.common.util.collection.MappingIterator.hasNext(MappingIterator.java:39)
	at java.base/java.util.Iterator.forEachRemaining(Iterator.java:132)
	at org.apache.hudi.client.utils.SparkMetadataWriterUtils.getBaseFileRecords(SparkMetadataWriterUtils.java:260)
	at org.apache.hudi.client.utils.SparkMetadataWriterUtils.readRecordsAsRows(SparkMetadataWriterUtils.java:232)
	at org.apache.hudi.client.utils.SparkMetadataWriterUtils.getExpressionIndexRecordsIterator(SparkMetadataWriterUtils.java:352)
	at org.apache.hudi.client.utils.SparkMetadataWriterUtils.lambda$getExprIndexRecords$69dbdb2c$1(SparkMetadataWriterUtils.java:320)
	at org.apache.hudi.data.HoodieJavaRDD.lambda$flatMap$a6598fcb$1(HoodieJavaRDD.java:160)
	at or

org.apache.hudi.exception.HoodieMetadataException: Failed to get expression index updates for partition expr_index_idx_column_ts

In [ ]:
// Data skipping would be performed using column stat expression index
spark.sql(s"SELECT uuid, rider FROM hudi_indexed_table WHERE from_unixtime(ts/1000, 'yyyy-MM-dd') = '2023-10-17'").show();
// Data skipping would be performed using bloom filter expression index
spark.sql(s"SELECT * FROM hudi_indexed_table WHERE driver = 'driver-N'").show();
// Data skipping would be performed using secondary index
spark.sql(s"SELECT * FROM hudi_indexed_table WHERE rider = 'rider-B'").show();

+----+-----+
|uuid|rider|
+----+-----+
+----+-----+

+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name| ts|uuid|rider|driver|fare|city|
+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+

+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name| ts|uuid|rider|driver|fare|city|
+-------------------+--------------------+------------------+----------------------+-----------------+---+----+-----+------+----+----+
+

In [ ]:
val command = """ls -lh "/tmp/hudi/hudi_indexed_table/*/""""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/city=chennai/:
total 428K
-rw-r--r-- 1 ryan ryan 426K Mar  3 22:57 4e67c3ef-8391-4fac-b346-7c39fb0ba705-0_2-31-57_20250303225703194.parquet

/tmp/hudi/hudi_indexed_table/city=san_francisco/:
total 856K
-rw-r--r-- 1 ryan ryan 427K Mar  3 22:57 dce0274a-5918-40fc-8ca2-ab126efb9431-0_0-104-262_20250303225731238.parquet
-rw-r--r-- 1 ryan ryan 427K Mar  3 22:57 dce0274a-5918-40fc-8ca2-ab126efb9431-0_0-31-55_20250303225703194.parquet

/tmp/hudi/hudi_indexed_table/city=sao_paulo/:
total 428K
-rw-r--r-- 1 ryan ryan 426K Mar  3 22:57 6723e66e-8b67-41c7-a9e0-81206c292231-0_1-31-56_20250303225703194.parquet



command: String = "ls -lh \"/tmp/hudi/hudi_indexed_table/*/\""
result: String = [lazy]

In [ ]:
val command = """ls -lh "/tmp/hudi/hudi_indexed_table/.hoodie/*/""""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/.hoodie/metadata/:
total 28K
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 column_stats
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 expr_index_idx_bloom_driver
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 expr_index_idx_column_ts
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 files
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 partition_stats
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 record_index
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 22:57 secondary_index_idx_rider

/tmp/hudi/hudi_indexed_table/.hoodie/timeline/:
total 48K
-rw-r--r-- 1 ryan ryan    0 Mar  3 22:57 20250303225703194.commit.requested
-rw-r--r-- 1 ryan ryan 2.6K Mar  3 22:57 20250303225703194.inflight
-rw-r--r-- 1 ryan ryan 3.4K Mar  3 22:57 20250303225703194_20250303225706901.commit
-rw-r--r-- 1 ryan ryan    0 Mar  3 22:57 20250303225707922.indexing.inflight
-rw-r--r-- 1 ryan ryan  830 Mar  3 22:57 20250303225707922.indexing.requested
-rw-r--r-- 1 ryan ryan 1.2K Mar  3 22:57 20250303225707922_20250303225709065.indexing


command: String = "ls -lh \"/tmp/hudi/hudi_indexed_table/.hoodie/*/\""
result: String = [lazy]

* But before that, I just noticed, index creations generated a lot of deltacommit.
* Most of the indexes are BULK_INSERT.
* Maybe because we are not adding or removing column, so the indexes are not updated (UPSERT_PREPPED).
* So the indexes are treated as a table.

In [ ]:
// First find all commits.
val command = """find /tmp/hudi/hudi_indexed_table/.hoodie/ -name "*commit"  """
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/.hoodie/timeline/20250303225703194_20250303225706901.commit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/20250303225703194_20250303225706873.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/20250303225714408_20250303230324483.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250303225704606.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/00000000000000001_20250303225704858.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/20250303225711016_20250303225714112.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/00000000000000003_20250303225708979.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/00000000000000002_20250303225705233.deltacommit
/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/20250303225712623_20250303225714896.deltacommit
/tmp/hudi/hudi_indexed_

command: String = "find /tmp/hudi/hudi_indexed_table/.hoodie/ -name \"*commit\"  "
result: String = [lazy]

In [13]:
import $ivy.`org.apache.spark:spark-avro_2.12:3.3.2`
val command = """find /tmp/hudi/hudi_indexed_table -name "*.deltacommit" -exec stat -c \"%y %s %n\" {} + | sort"""
lazy val result = Seq("bash", "-c", s"eval $command").!!
lazy val hudiMetas = result.split("\n")
for(hudiMeta <- hudiMetas){
    val path = hudiMeta.split(" ")(4)
    println(path)
    spark.read.format("avro").load(path).select("operationType", "partitionToWriteStats", "extraMetadata", "version").show(truncate=false)
    println()
}

/tmp/hudi/hudi_indexed_table/.hoodie/metadata/.hoodie/timeline/00000000000000000_20250303231429171.deltacommit
+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import $ivy.$
command: String = "find /tmp/hudi/hudi_indexed_table -name \"*.deltacommit\" -exec stat -c \\\"%y %s %n\\\" {} + | sort"
result: String = [lazy]
hudiMetas: Array[String] = [lazy]

* Unfortunately, the hfile files are not formatted files (csv, avro, parquet), so we can't just simply observe them.
* Homework: read Hudi source code so we can know how to decode the hfile.
* Hint (maybe): org.apache.hudi.io.storage.HoodieHBaseKVComparator is in the file. Maybe start from there?

In [14]:
// Drop all the indexes
spark.sql(s"SHOW INDEXES FROM hudi_indexed_table").show();
spark.sql(s"DROP INDEX secondary_index_idx_rider on hudi_indexed_table");
spark.sql(s"DROP INDEX record_index on hudi_indexed_table");
spark.sql(s"DROP INDEX expr_index_idx_bloom_driver on hudi_indexed_table");
spark.sql(s"DROP INDEX expr_index_idx_column_ts on hudi_indexed_table");
// No indexes should show up for the table
spark.sql(s"SHOW INDEXES FROM hudi_indexed_table").show();

spark.sql(s"SET hoodie.metadata.record.index.enable=false");

+--------------------+---------------+----------+
|          index_name|       col_name|index_type|
+--------------------+---------------+----------+
|        record_index|   record_index|          |
|        column_stats|   column_stats|          |
|     partition_stats|partition_stats|          |
|secondary_index_i...|secondary_index|     rider|
|expr_index_idx_co...|   column_stats|        ts|
|expr_index_idx_bl...|  bloom_filters|    driver|
+--------------------+---------------+----------+

+---------------+---------------+----------+
|     index_name|       col_name|index_type|
+---------------+---------------+----------+
|   column_stats|   column_stats|          |
|partition_stats|partition_stats|          |
+---------------+---------------+----------+



res14_1: DataFrame = []
res14_2: DataFrame = []
res14_3: DataFrame = []
res14_4: DataFrame = []
res14_6: DataFrame = [key: string, value: string]

In [15]:
val command = """ls -lh "/tmp/hudi/hudi_indexed_table/*/""""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/city=chennai/:
total 428K
-rw-r--r-- 1 ryan ryan 426K Mar  3 23:14 5fb66e2c-4ca9-49b3-beac-6144f80a9ec4-0_2-31-57_20250303231427781.parquet

/tmp/hudi/hudi_indexed_table/city=san_francisco/:
total 856K
-rw-r--r-- 1 ryan ryan 427K Mar  3 23:14 75a7500e-714d-411e-97a2-952b2bdec2c0-0_0-104-262_20250303231440855.parquet
-rw-r--r-- 1 ryan ryan 427K Mar  3 23:14 75a7500e-714d-411e-97a2-952b2bdec2c0-0_0-31-55_20250303231427781.parquet

/tmp/hudi/hudi_indexed_table/city=sao_paulo/:
total 428K
-rw-r--r-- 1 ryan ryan 426K Mar  3 23:14 8f38470a-e784-45b6-a4fd-5d1fb705ac19-0_1-31-56_20250303231427781.parquet



command: String = "ls -lh \"/tmp/hudi/hudi_indexed_table/*/\""
result: String = [lazy]

In [16]:
val command = """ls -lh "/tmp/hudi/hudi_indexed_table/.hoodie/*/""""
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/.hoodie/metadata/:
total 16K
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 23:14 column_stats
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 23:14 files
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 23:14 partition_stats
drwxr-xr-x 2 ryan ryan 4.0K Mar  3 23:20 record_index

/tmp/hudi/hudi_indexed_table/.hoodie/timeline/:
total 48K
-rw-r--r-- 1 ryan ryan    0 Mar  3 23:14 20250303231427781.commit.requested
-rw-r--r-- 1 ryan ryan 2.6K Mar  3 23:14 20250303231427781.inflight
-rw-r--r-- 1 ryan ryan 3.4K Mar  3 23:14 20250303231427781_20250303231431459.commit
-rw-r--r-- 1 ryan ryan    0 Mar  3 23:14 20250303231432571.indexing.inflight
-rw-r--r-- 1 ryan ryan  830 Mar  3 23:14 20250303231432571.indexing.requested
-rw-r--r-- 1 ryan ryan 1.2K Mar  3 23:14 20250303231432571_20250303231433702.indexing
-rw-r--r-- 1 ryan ryan    0 Mar  3 23:14 20250303231436030.indexing.inflight
-rw-r--r-- 1 ryan ryan  827 Mar  3 23:14 20250303231436030.indexing.requested
-rw-r--r-- 1 ryan ryan 1.2K Mar  3 23:14 202503

command: String = "ls -lh \"/tmp/hudi/hudi_indexed_table/.hoodie/*/\""
result: String = [lazy]

### Time Travel Query

In [19]:
val updatesDf = spark.read.format("hudi").load(basePath).filter($"rider" === "rider-D").withColumn("fare", col("fare") * 10)

updatesDf.write.format("hudi").
  option("hoodie.datasource.write.operation", "upsert").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  mode(Append).
  save(basePath)

updatesDf: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [20]:
val command = """find /tmp/hudi/hudi_indexed_table/.hoodie/ -name "*.commit" """
lazy val result = Seq("bash", "-c", s"eval $command").!!
println(result)

/tmp/hudi/hudi_indexed_table/.hoodie/timeline/20250303231427781_20250303231431459.commit
/tmp/hudi/hudi_indexed_table/.hoodie/timeline/20250303232318387_20250303232319507.commit



command: String = "find /tmp/hudi/hudi_indexed_table/.hoodie/ -name \"*.commit\" "
result: String = [lazy]

In [ ]:
spark.read.format("hudi").
  option("as.of.instant", "20250303231427781").
  load(basePath).show()

spark.read.format("hudi").
  option("as.of.instant", "2025-03-03 23:23:18.387").
  load(basePath).show()

// It is equal to "as.of.instant = 2025-03-04 00:00:00"
spark.read.format("hudi").
  option("as.of.instant", "2025-03-04").
  load(basePath).show()


+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250303231427781|20250303231427781...|e96c4396-3fad-413...|    city=san_francisco|75a7500e-714d-411...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M| 27.7|san_francisco|
|  20250303231427781|20250303231427781...|9909a8b1-2d15-4d3...|    city=san_francisco|75a7500e-714d-411...|1695046462179|9909a8b1-2d15-4d3...|rider-D|driver-L| 33.9|san_francisco|
|  20250303231427781|20250303231427781...|334e26e9-8355-45c...|    city=san_francisco|75a7500e-714d-

In [27]:
spark.stop()